## Prepare the workspace

In [ ]:
import os

# Cập nhật biến môi trường PATH với các đường dẫn mới
os.environ['PATH'] += ":/root/.local/bin"
os.environ['PATH'] += ":/opt/conda/lib/python3.6/site-packages"

In [ ]:
# !python -m pip install torch==1.11.0
!pip install --upgrade pip
!pip install torchvision==0.12.0
!pip3 install numpy --pre torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/nightly/cu117

In [ ]:
# Check PyTorch version and CUDA availability if GPU is enabled.
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")  # Returns True if GPU is enabled

# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications.

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below.

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
import torch
import torch.nn as nn
from torchvision import models
from collections import OrderedDict
import argparse
from torchvision import transforms
from PIL import Image
import json
from model import create_model
import argparse
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from model import create_model

## Load the data

You will utilize torchvision to load the dataset. For detailed guidance, refer to the ([official documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The dataset should be present in the same directory as this notebook. If it's missing, you can [download it from here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz).

If the flowers/ dataset is not located in the current directory **/workspace/home/aipnd-project/**, you can retrieve it by running the following commands:

```bash
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!rm -rf flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers
```


In [19]:
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!rm -rf flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers

## Data Description
The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.


In [20]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# Define image transformations for training, validation, and testing datasets
train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

valid_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load datasets with transformations applied using ImageFolder
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=valid_transform)

# Define DataLoaders for each dataset, with batching and shuffling enabled
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=64, shuffle=True)


In [ ]:
# Get one batch from the test loader
batch_iterator = iter(test_loader)
batch_images, batch_labels = next(batch_iterator)

# Choose the first image and its label from the batch
first_image = batch_images[0]
first_label = batch_labels[0]

# Convert the image to a NumPy array for visualization
image_array = first_image.numpy()

# Adjust the image dimensions for proper display
adjusted_image = np.transpose(image_array, (1, 2, 0))

# Display the image with its label in the plot title
plt.imshow(adjusted_image)
plt.title(f"Class: {first_label.item()}")
plt.show()


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

## Note for Workspace users:
If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
def build_model(backbone='vgg16'):

    if backbone == 'vgg16':
        model = models.vgg16(pretrained=True)
    elif backbone == 'densenet121':
        model = models.densenet121(pretrained=True)
    else:
        raise ValueError("Backbone model not supported")

    # Freeze the parameters of the pre-trained model
    for param in model.parameters():
        param.requires_grad = False

    # Replace the classifier with a custom one
    model.classifier = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(25088, 2048)),
        ('relu', nn.ReLU()),
        ('dropout1', nn.Dropout(0.3)),
        ('fc2', nn.Linear(2048, 256)),
        ('dropout2', nn.Dropout(0.3)),
        ('relu2', nn.ReLU()),
        ('fc3', nn.Linear(256, 102)),
        ('output', nn.LogSoftmax(dim=1))
    ]))

    return model

# Set up the device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and move it to the device
model = build_model().to(device)

# Define the loss function
loss_fn = nn.NLLLoss()

# Use Adam optimizer for the classifier parameters
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

print("-----------< Model Summary >-----------")
print(model)
print("--------------------------------------< Training Start >--------------------------------------")

# Define the training process
epochs = 3
steps = 0
loss_accumulated = 0
print_every = 10
train_loss_history, val_loss_history = [], []

for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1

        # Move inputs and labels to the appropriate device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output_probs = model(inputs)
        loss = loss_fn(output_probs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        loss_accumulated += loss.item()

        if steps % print_every == 0:
            val_loss = 0
            accuracy = 0
            model.eval()

            with torch.no_grad():
                for inputs, labels in validloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    output_probs = model(inputs)
                    batch_loss = loss_fn(output_probs, labels)
                    val_loss += batch_loss.item()

                    # Calculate accuracy
                    predicted_probs = torch.exp(output_probs)
                    top_prob, top_class = predicted_probs.topk(1, dim=1)
                    correct = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(correct.type(torch.FloatTensor)).item()

            train_loss_history.append(loss_accumulated / print_every)
            val_loss_history.append(val_loss / len(validloader))
            print(f"Epoch: {epoch+1}/{epochs}.. "
                  f"Train Loss: {loss_accumulated/print_every:.3f}.. "
                  f"Val Loss: {val_loss/len(validloader):.3f}.. "
                  f"Val Accuracy: {accuracy/len(validloader):.3f}")
            loss_accumulated = 0
            model.train()

print("--------------------------------------< Training Finished >--------------------------------------")


In [ ]:
param_range = np.arange(1, 10)

# Plot the training and validation losses
plt.plot(train_loss_history, label='Training Loss')
plt.plot(val_loss_history, label='Validation Loss')

# Customize the legend to improve the plot presentation
plt.legend(loc='upper right', frameon=False)

plt.show()


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# Evaluate the model on the test set
test_loss = 0
total_accuracy = 0
model.eval()  # Switch to evaluation mode
with torch.no_grad():  # Disable gradient calculation for testing
    for inputs, labels in testloader:
        # Move data to the device (GPU or CPU)
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass through the model
        predictions = model(inputs)
        loss = criterion(predictions, labels)

        # Accumulate the loss
        test_loss += loss.item()

        # Compute accuracy
        probabilities = torch.exp(predictions)
        top_probability, predicted_class = probabilities.topk(1, dim=1)
        correct = predicted_class == labels.view(*predicted_class.shape)
        total_accuracy += torch.mean(correct.type(torch.FloatTensor)).item()

# Print test results
print(f"Test Loss: {test_loss/len(testloader):.3f}.. ", 
      f"Test Accuracy: {total_accuracy/len(testloader):.3f}")


Test Loss: 0.618..  Test Accuracy: 0.826


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Save the model checkpoint
checkpoint_path = 'model_checkpoint.pth'

# Store model-related details in a dictionary
checkpoint_data = {
    'input_dimensions': 25088,
    'output_classes': 102,
    'model_architecture': 'vgg16',
    'learning_rate': 0.001,
    'training_epochs': epochs,
    'optimizer_state': optimizer.state_dict(),
    'model_state': model.state_dict(),
    'class_index_mapping': train_data.class_to_idx
}

# Save the checkpoint
torch.save(checkpoint_data, checkpoint_path)


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# Load the checkpoint from the saved file
loaded_checkpoint = torch.load(checkpoint_path)

# Initialize the model with the same architecture
loaded_model = Net()

# Map the class indices from the checkpoint to the model
loaded_model.class_to_idx = loaded_checkpoint['class_index_mapping']

# Load the model's state dictionary to restore the trained parameters
loaded_model.load_state_dict(loaded_checkpoint['model_state'])

# Print the model structure
print(loaded_model)


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network.

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training.

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation.

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    transform = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.5, 0.5, 0.5],
                                                              [0.5, 0.5, 0.5])])
    imgp = Image.open(image)
    return transform(imgp)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(tensor, ax=None, title=None):
    """Display a tensor as an image."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # Convert the tensor from (C, H, W) to (H, W, C) for matplotlib compatibility
    image_array = tensor.numpy().transpose((1, 2, 0))
    
    # Revert normalization applied during preprocessing
    mean_values = np.array([0.485, 0.456, 0.406])
    std_values = np.array([0.229, 0.224, 0.225])
    image_array = std_values * image_array + mean_values
    
    # Clip the image values to the range [0, 1] to avoid display issues
    image_array = np.clip(image_array, 0, 1)
    
    ax.imshow(image_array)
    
    if title:
        ax.set_title(title)
    
    return ax


## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
import torch
import numpy as np

def predict(image_path, model, topk=5):
    ''' Make a prediction for an image using a trained deep learning model.

    Args:
        image_path (str): Path to the image file to be classified.
        model (torch.nn.Module): The pre-trained model used for classification.
        topk (int): Number of top predicted classes to return.

    Returns:
        top_probabilities (Tensor): Probabilities of the top predicted classes.
        top_classes (Tensor): Indices of the top predicted classes.
    '''
    # Set the model to evaluation mode and use the appropriate device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    # Prepare the image and convert it into a tensor for the model
    processed_image = process_image(image_path)  # Ensure process_image is defined
    processed_image = processed_image.unsqueeze(0)  # Add batch dimension
    processed_image = processed_image.to(device, dtype=torch.float32)

    # Get predictions from the model
    with torch.no_grad():
        log_probabilities = model(processed_image)

    # Compute the actual probabilities and retrieve the topk predictions
    probabilities = torch.exp(log_probabilities)
    top_probabilities, top_class_indices = probabilities.topk(topk, dim=1)

    return top_probabilities, top_class_indices


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# Display an image with its top 5 predicted classes
image_file = test_dir + "/100/image_07896.jpg"
probabilities, class_indices = make_prediction(image_file, model)

# Print the predicted probabilities and corresponding class indices
print("Predicted Probabilities:", probabilities)
print("Predicted Class Indices:", class_indices)


In [ ]:
def display_prediction(image_file, model, topk=5, category_to_name=cat_to_name):
    ''' Display the image and its topk predicted categories with their probabilities.

    Args:
        image_file (str): Path to the image you want to predict and show.
        model (torch.nn.Module): Pre-trained deep learning model for prediction.
        topk (int): Number of top categories to display with probabilities.

    Returns:
        None
    '''
    # Set up the figure with two subplots for image and bar chart
    plt.rcdefaults()

    fig, (axis1, axis2) = plt.subplots(1, 2, figsize=(10, 5))

    # Get predictions and image
    prediction_scores = predict(image_file, model, topk)
    img = process_image(image_file)

    # Show the image on the first subplot (axis1)
    imshow(img, ax=axis1)
    axis1.axis('off')
    label_index = 1  # Set to the appropriate index based on prediction
    axis1.set_title(category_to_name[str(label_index)])  # Set title to the predicted category name

    # Prepare data for the bar chart on the second subplot (axis2)
    probabilities = np.array(prediction_scores[0][0].cpu())
    class_names = [category_to_name[str(label_index+1)] for label_index in np.array(prediction_scores[1][0].cpu())]

    y_positions = np.arange(topk)
    axis2.set_yticks(y_positions)
    axis2.set_yticklabels(class_names)
    axis2.set_xlabel('Probability')
    axis2.invert_yaxis()
    axis2.barh(y_positions, probabilities)

    # Display the plot
    plt.show()

# Call the function with the image path and model
show_prediction(image_file=image_path, model=model)


## Reminder for Workspace users
If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again.
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.

In [ ]:
import os

# Define the path to the .pth file
model_path = filepath  # Assign the file path you wish to check

# Check if the file exists
if os.path.isfile(model_path):
    try:
        # Attempt to remove the file
        os.remove(model_path)
        print(f"The file at {model_path} has been successfully deleted.")
    except OSError as error:
        print(f"An error occurred: {error}")
else:
    print(f"File {model_path} was not found.")


checkpoint_model.pth has been deleted.
